In [1]:
from os.path import exists
import pandas as pd
from subprocess import check_output
import pickle
from IPython.display import display, clear_output

def scan(ip, port):
    cmd = ['nmap', '-sU', '-p', str(port), ip]
    display(' '.join(cmd))
    return check_output(cmd)


def parse_result(lines):
    lines = lines.decode('utf-8').split('\n')
    for r, line in enumerate(lines):
        if line.startswith('PORT'):
            break
    else:
        raise ValueError("No Result found!")
    keys = lines[r].split()
    vals = lines[r+1].split()
    return pd.Series(data=vals, index=keys)


tello_ip  = '192.168.10.1'
output_csv = "port-scan.csv"
output_pkl = "port-scan.pkl"

In [15]:
port_range = range(1024, 2**16, 1)

In [16]:
if exists(output_pkl):
    with open(output_pkl, 'rb') as fo:
        finished_scans = pickle.load(fo)
else:
    finished_scans = {}

In [40]:
for port in port_range:
    if port not in finished_scans:
        clear_output()
        try:
            scan_output = parse_result(scan(tello_ip, port))
            finished_scans[port] = scan_output
        except Exception as e:
            print(e)
            break
    else:
        print(port, 'already scanned.  continuing..')

nmap -sU -p 1424 192.168.10.1
No Result found!


In [41]:
with open(output_pkl, 'wb') as fo:
    pickle.dump(finished_scans, fo)

Prepare a pandas Dataframe of the scanned ports

In [42]:
df = pd.concat(finished_scans)
df.index.names = ['Port', 'Attrib']
df = df.unstack('Attrib')
df.head()

Attrib,PORT,STATE,SERVICE
Port,,,
1024,1024/udp,closed,unknown
1025,1025/udp,closed,blackjack
1026,1026/udp,closed,win-rpc
1027,1027/udp,closed,unknown
1028,1028/udp,closed,ms-lsa


Print all ports that are not closed

In [43]:
df.loc[df['STATE'] != 'closed']

Attrib,PORT,STATE,SERVICE
Port,,,
1065,1065/udp,open|filtered,syscomlan
1103,1103/udp,open|filtered,adobeserver-2
1127,1127/udp,open|filtered,unknown
1132,1132/udp,open|filtered,unknown
1135,1135/udp,open|filtered,unknown
1163,1163/udp,open|filtered,sddp
1189,1189/udp,open|filtered,unet
1221,1221/udp,open|filtered,unknown
1256,1256/udp,open|filtered,de-server
